In [4]:
import pickle
import numpy as np

In [2]:
# Copy the the file form the results folder of the experiment you want to analyze
with open('nonnested_cval_mltsatt_l819.pkl', 'rb') as f:
    results = pickle.load(f)

In [6]:
# Take the mean and std of our metrics acrross the 10 folds
roc = []
fp = []
for i in range(10):
    roc.append(results[i]["roc"])
    fp.append(results[i]["fp"])


mean_roc = np.array(roc).mean()
std_roc = np.array(roc).std()
mean_fp = np.array(fp).mean()
std_fp = np.array(fp).std()

print("ROC: ", mean_roc, " +/- ", std_roc)
print("FP: ", mean_fp, " +/- ", std_fp)

ROC:  0.6035389610389611  +/-  0.11516134094149343
FP:  0.35857142857142854  +/-  0.11073962843590567
